In [1]:
import functools
import numpy
import pandas
import sklearn.linear_model
from sympy import *
from fractions import Fraction
import xicor.xicor


In [2]:
# define xicor as a funciton
def xicor_fn(*, tp_c: int, fp_c: int, tn_c: int, fn_c: int, n_reps=100):
    """
    Define xicor as a function of observed counts.

    :param tp_c: true positive count
    :param fp_c: false positive count
    :param tn_c: true negative count
    :param fn_c: false negative count
    :return: estimated expected xicor correlation
    """
    assert tp_c >= 0
    assert fp_c >= 0
    assert tn_c >= 0
    assert fn_c >= 0
    assert tp_c + fn_c > 0  # y has positive examples
    assert tn_c + fp_c > 0  # y has negative examples
    x = [1] * tp_c + [1] * fp_c + [0] * tn_c + [0] * fn_c
    y = [1] * tp_c + [0] * fp_c + [0] * tn_c + [1] * fn_c
    evals = [xicor.xicor.Xi(x, y).correlation for rep in range(n_reps)]
    return numpy.mean(evals)

In [3]:
# estimate of expected value of xicor formula for x, y both 0/1
# https://arxiv.org/abs/1909.10140
# xicor is
# 1 - n sum(i=1,...,n-1) |r(i+1) - r(i)| / (2 sum(i=1,...,n) l(i) (n - l(i))
# n = tp + fp + tn + fn
# by the tie-breaking rules in https://CRAN.R-project.org/package=XICOR calculateXI() we have:
# ri is always tn + fp or n, so its abs differences are always 0 or tp + fn
# li is always n or tp + fn, so n - li is always 0 or tn + fp
# for the first tn + fn terms of the r(i) sum we have
# E[|r(i+1) - r(i)|] ~ |non-zero-diff| * prob_diff_is_non_zero
#                    = (tp + fn) * 2 * tn * fn / (tn + fn)**2
# for the next tp + fp terms of the r(i) sum we have
# E[|r(i+1) - r(i)|] ~ |non-zero-diff| * prob_diff_is_non_zero
#                    = (tp + fn) * 2 * tp * fp / (tp + fp)**2
# and the denominator is just 2 (tp + fp) * (tp + fp) * (tn + fn)
# putting this all together
tp, fp, tn, fn = symbols('tp fp tn fn')
xi_cor_estimate = (
    1 - (tp + fp + tn + fn) * (
        (tn + fn) * (tp + fn) * 2 * tn * fn / (tn + fn)**2
        + (tp + fp) * (tp + fn) * 2 * tp * fp / (tp + fp)**2
    ) / (2 * (tp + fn) * (tn + fp) * (tp + fn))
)

xi_cor_estimate = xi_cor_estimate.simplify()
xi_cor_estimate

def xicor_rat_fn(*, tp_c: int, fp_c: int, tn_c: int, fn_c: int):
    """
    Estimate expected value of xicor using rational function.

    :param tp_c: true positive count
    :param fp_c: false positive count
    :param tn_c: true negative count
    :param fn_c: false negative count
    :return: xicor correlation
    """
    return xi_cor_estimate.subs({tp: tp_c, tn: tn_c, fp: fp_c, fn: fn_c})

In [4]:
float(xicor_rat_fn(tp_c=100, fp_c=10, tn_c=50, fn_c=20))

0.4155844155844156

In [5]:
xicor_fn(tp_c=100, fp_c=10, tn_c=50, fn_c=20)

0.414375

In [6]:
# check symmetries
a, b, c, d, s = symbols('a b c d s')

# scale invariant (should equal 0)
(xicor_rat_fn(tp_c=s*a, fp_c=s*b, tn_c=s*c, fn_c=s*d) - xicor_rat_fn(tp_c=a, fp_c=b, tn_c=c, fn_c=d)).simplify()

0

In [7]:
# gets independent case right (should equal 0)
xicor_rat_fn(tp_c=s*a, fp_c=s*b, tn_c=b, fn_c=a).simplify()

0

In [8]:
# gets perfect case right (should be 1)
xicor_rat_fn(tp_c=a, fp_c=0, tn_c=c, fn_c=0).simplify()

1

In [9]:
# gets flipping y right (should equal 0)
(xicor_rat_fn(tp_c=b, fp_c=a, tn_c=d, fn_c=c) - xicor_rat_fn(tp_c=a, fp_c=b, tn_c=c, fn_c=d)).simplify()

0

In [10]:
# gets flipping x right (should equal 0)
(xicor_rat_fn(tp_c=c, fp_c=d, tn_c=a, fn_c=b) - xicor_rat_fn(tp_c=a, fp_c=b, tn_c=c, fn_c=d)).simplify()


0

In [11]:
# is assymetric in x and y (doesn't equal 0)
(xicor_rat_fn(tp_c=a, fp_c=c, tn_c=b, fn_c=d) - xicor_rat_fn(tp_c=a, fp_c=b, tn_c=c, fn_c=d)).subs(
    {a: 1, b: 2, c: 3, d:4})

5/42